In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import os
import io
import hashlib
from PIL import Image
import requests
import pandas as pd
import time

In [2]:
# To read from excel worksheet
# eg. test_sheet = pd.read_excel(r"C:\path\to\excelsheet.xlsx", sheet_name='Sheet1')

# To write to excel worksheet
# eg. test_sheet.to_excel(r'C:\path\to\excelsheet.xlsx', index = False)

# Read all files in directory and returns them as a Python array.
# eg. files = list_file_names(r"C:\Users\maeky\OneDrive\Desktop\library\Projects\Ear Dataset\testimages")
# File format: Subject_SkinType_Gender_Occlusion_Pitch_Roll_Yaw_LicenseURL.jpg
# allso add count to end of subject label
def list_file_names(path):
    arr = []
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            arr.append(file)
    return arr

def add_images(files, xlsheet):
    """
    files: list of images obtained using list_file_names function
    eg. test_sheet = add_images(fileNames, test_sheet)
    """
    for file in files:
        row_to_add = {}
        file = file.split('_')
        
        file[0] = f"{file[0]} {file[1]}"
        row_to_add[xlsheet.columns[0]] = file[0]
        file[2] = int(file[2])
        
        for x in range(1, len(xlsheet.columns)):
            row_to_add[xlsheet.columns[x]] = file[x+1]
        xlsheet = xlsheet.append(row_to_add, ignore_index=True)
    return xlsheet

def remove_images(no_of_images, xlsheet):
    """
    eg. remove_images(1, test_sheet)
    """
    for _ in range(no_of_images):
        xlsheet.drop([len(xlsheet)-1], inplace=True)
        
def remove_this(column_name, to_remove, xlsheet):
    """
    remove row at specified column for entry errors
    eg. test_sheet = remove_this('Subject', 'Test two', test_sheet)
    """
    remove = xlsheet.index[xlsheet[column_name] == to_remove].tolist()
    if len(remove) > 0:
        count = 1
        for i in range(len(remove)):
            confirm = input(f"Remove this?\n{xlsheet.iloc[remove[i]]}\n")
            if confirm == 'y' or confirm == 'yes': 
                xlsheet = xlsheet.drop([remove[i]]).reset_index(drop=True)
                if i+1<len(remove):
                    remove[i+1] -= count
                    count += 1
    else: print(f"File does not exist.")
    return xlsheet

In [3]:
test_sheet = pd.read_excel(r"C:\Users\maeky\OneDrive\Desktop\library\Projects\Ear Dataset\testmain.xlsx", sheet_name='Sheet1')
test_sheet

,Subject,Skintone,Gender,Occlusion,Pitch,Yaw,Roll,LiscenseURL
0,Zayn Malik,2,M,0,1,2,0,https://pyxis.nymag.com/v1/imgs/f0d.jpg
1,Beyonce Knowles,4,F,1,2,0,1,https://pyxis.nymag.com/v1/imgs/f1d.jpg
2,Jeon Jung-kook,1,M,2,0,1,2,https://pyxis.nymag.com/v1/imgs/f0d.jpg
3,Kanye West,6,M,0,1,2,0,https://pyxis.nymag.com/v1/imgs/f3d.jpg


In [4]:
fileNames = list_file_names(r"C:\Users\maeky\OneDrive\Desktop\library\Projects\Ear Dataset\testimages")
fileNames

['Test_one_1_F_2_0_1_2_httpslicenseurlimgs1.jpg',
 'Test_three_3_F_____httpslicenseurlimgs3.jpg',
 'Test_two_2_M_1_2_0_1_httpslicenseurlimgs2.jpg']

In [5]:
fileNames = list_file_names(r"C:\Users\maeky\OneDrive\Desktop\library\Projects\Ear Dataset\testimages")
test_sheet = add_images(fileNames, test_sheet)
test_sheet = add_images(fileNames, test_sheet)
remove_images(2, test_sheet)
test_sheet = remove_this('Subject', 'Test one', test_sheet)
test_sheet

Remove this?
Subject                        Test one
Skintone                              1
Gender                                F
Occlusion                             2
Pitch                                 0
Yaw                                   1
Roll                                  2
LiscenseURL    httpslicenseurlimgs1.jpg
Name: 4, dtype: object
yes
Remove this?
Subject                        Test one
Skintone                              1
Gender                                F
Occlusion                             2
Pitch                                 0
Yaw                                   1
Roll                                  2
LiscenseURL    httpslicenseurlimgs1.jpg
Name: 6, dtype: object
no


,Subject,Skintone,Gender,Occlusion,Pitch,Yaw,Roll,LiscenseURL
0,Zayn Malik,2,M,0,1,2,0,https://pyxis.nymag.com/v1/imgs/f0d.jpg
1,Beyonce Knowles,4,F,1,2,0,1,https://pyxis.nymag.com/v1/imgs/f1d.jpg
2,Jeon Jung-kook,1,M,2,0,1,2,https://pyxis.nymag.com/v1/imgs/f0d.jpg
3,Kanye West,6,M,0,1,2,0,https://pyxis.nymag.com/v1/imgs/f3d.jpg
4,Test three,3,F,,,,,httpslicenseurlimgs3.jpg
5,Test two,2,M,1,2,0,1,httpslicenseurlimgs2.jpg
6,Test one,1,F,2,0,1,2,httpslicenseurlimgs1.jpg


In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    # Helper function for search_and_download function
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,url:str):
    # Helper function for search_and_download function
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    # search_term is the term to be put into the google search bar
    # driver_path is local chromedriver path
    # target_path is path to folder to be saved in
    # number_images is the number of images to be saved
    
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
DRIVER_PATH = r"C:\Users\maeky\Downloads\Installs\Scraping\chromedriver"
path = r"C:\Users\maeky\OneDrive\Desktop\library\Projects\Ear Dataset\images"
search_and_download('Zayn Left Ear', DRIVER_PATH, path, 10)

C:\Users\maeky\AppData\Local\Temp/ipykernel_20752/4045343190.py:86: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
